In [ ]:
from google.colab import files

# Upload file kaggle.json yang baru didownload
files.upload()

# Pindahkan ke folder yang benar agar bisa dibaca sistem
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
# Download dataset law-indonesia
!kaggle datasets download -d hermansugiharto/law-indonesia

# Unzip datanya
!unzip law-indonesia.zip

Streaming output truncated to the last 5000 lines.
  inflating: PP/1961/Peraturan Pemerintah Nomor 227 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 228 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 229 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 23 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 230 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 231 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 232 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 234 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 235 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 236 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 237 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 239 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah Nomor 24 Tahun 19610.pdf  
  inflating: PP/1961/Peraturan Pemerintah

In [ ]:
import pandas as pd
import json
import glob

# 1. Cek file CSV apa yang ada
csv_files = glob.glob("*.csv")
print(f"File CSV ditemukan: {csv_files}")

# Anggap kita pakai file pertama yang ditemukan (biasanya 'indo_law.csv' atau sejenisnya)
# Ganti nama file ini jika berbeda hasil print di atas
filename = csv_files[0]
df = pd.read_csv(filename)

# Tampilkan 5 data teratas untuk pengecekan kolom
print("\nContoh Data Mentah:")
print(df.head())

# --- FUNGSI TRANSFORMASI DATA ---
# Kita harus mapping kolom CSV ke format Instruction.
# Berdasarkan df.head(), kita akan menggunakan 'Title' sebagai nama UU dan 'Tentang' sebagai isi/ringkasan

formatted_data = []

print("\nSedang memproses data...")

# Iterasi setiap baris data
for index, row in df.iterrows():
    # Pastikan data 'Tentang' tidak kosong untuk dijadikan konten
    if pd.isna(row.get('Tentang', None)) or str(row.get('Tentang', '')).strip() == '':
        continue

    # -----------------------------------------------------------
    # LOGIKA PEMBUATAN PERTANYAAN (Prompt Engineering)
    # Kita buat model seolah-olah ditanya oleh user
    # -----------------------------------------------------------

    # Ambil data dari kolom yang tersedia
    nama_dokumen = str(row.get('Title', 'Dokumen Hukum Tidak Dikenal'))
    isi_singkat_dokumen = str(row.get('Tentang', ''))

    # Template Pertanyaan 1: Tanya ringkasan dokumen
    # Contoh: "Apa isi dari Undang-Undang Dasar Republik Indonesia Tahun 1945?"
    instruction_1 = f"Apa ringkasan atau pokok bahasan dari {nama_dokumen}?"

    # Template Pertanyaan 2: Tanya dasar hukum (Variasi agar model pintar)
    # Contoh: "Jelaskan aturan mengenai X di UU Y"
    # Kita pakai chunk awal teks dokumen sebagai 'hint'
    snippet = isi_singkat_dokumen[:50] + ("..." if len(isi_singkat_dokumen) > 50 else "")
    instruction_2 = f"Jelaskan mengenai '{snippet}' yang diatur dalam {nama_dokumen}."

    # --- FORMAT UNTUK UNSLOTH ---
    # Kita masukkan ke list

    # Entry 1
    formatted_data.append({
        "instruction": instruction_1,
        "input": "", # Kosongkan input karena konteks sudah jelas di instruction
        "output": f"Berdasarkan {nama_dokumen}, pokok bahasan utamanya adalah: {isi_singkat_dokumen}"
    })

    # Entry 2 (Opsional, untuk memperbanyak variasi data)
    formatted_data.append({
        "instruction": instruction_2,
        "input": "",
        "output": isi_singkat_dokumen
    })

# Simpan ke format JSONL (Format standard HuggingFace/Unsloth)
output_file = "dataset_hukum_siap_training.json"
with open(output_file, 'w') as f:
    json.dump(formatted_data, f, indent=2)

print(f"\nSUKSES! {len(formatted_data)} data siap latih tersimpan di '{output_file}'")

File CSV ditemukan: ['list.csv']

Contoh Data Mentah:
                                               Title  wish_bt  \
0  Undang-Undang Dasar Republik Indonesia Tahun 1945     1945   
1  Ketetapan Majelis Permusyawaratan RakyatIV/MPR...     2004   
2  Ketetapan Majelis Permusyawaratan RakyatVI/MPR...     2000   
3  Ketetapan Majelis Permusyawaratan RakyatVII/MP...     2000   
4  Ketetapan Majelis Permusyawaratan RakyatIV/MPR...     2000   

                                         wrapper_URL  \
0                https://peraturan.go.id/id/uud-1945   
1  https://peraturan.go.id/id/tap-mpr-no-iv-mpr-1...   
2  https://peraturan.go.id/id/tap-mpr-no-vi-mpr-2...   
3  https://peraturan.go.id/id/tap-mpr-no-vii-mpr-...   
4  https://peraturan.go.id/id/tap-mpr-no-iv-mpr-2...   

                                             wrapper  \
0  Undang-undang Dasar Negara Republik Indonesia ...   
1    Garis-garis Besar Haluan Negara Tahun 1999-2004   
2  Pemisahan Tentara Nasional Indonesia Dan Kepol.

In [ ]:
from huggingface_hub import login
from transformers import AutoTokenizer
from datasets import load_dataset

# 1. LOGIN MANUAL (Agar tidak error 401 Unauthorized)
# Saat dijalankan, akan muncul kotak input. Paste Token HF Anda (Starts with 'hf_...') lalu Enter.
login()

# 2. LOAD TOKENIZER YANG BENAR (LLAMA-3)
# Jangan pakai Mistral jika kita melatih Llama-3.
model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit"
print(f"Sedang memuat tokenizer untuk: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
# Fix padding token agar tidak error saat training
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 3. LOAD & FORMAT DATASET
dataset = load_dataset("json", data_files="dataset_hukum_siap_training.json", split="train")

# Template Prompt (Alpaca)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Tambahkan EOS token di akhir
        text = alpaca_prompt.format(instruction, input, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

# Terapkan formatting
dataset = dataset.map(formatting_prompts_func, batched = True)

print("\n✅ SUKSES! Dataset siap & Tokenizer Llama-3 aktif.")
print("Contoh data pertama:", dataset[0]['text'][:100], "...")

Sedang memuat tokenizer untuk: unsloth/llama-3-8b-Instruct-bnb-4bit


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Map:   0%|          | 0/114988 [00:00<?, ? examples/s]


✅ SUKSES! Dataset siap & Tokenizer Llama-3 aktif.
Contoh data pertama: Below is an instruction that describes a task, paired with an input that provides further context. W ...
